In [ ]:
import pandas as pd
from nltk import word_tokenize
from nltk.probability import FreqDist

# Assuming df is your DataFrame and 'tag' is a column in df
# containing genre information, and 'lyrics_string_fdist' is another column

# Create a dictionary to store data for each genre and an 'all' category
genre_data = {'all': {'lyrics': '', 'tokens': []}}

# Iterate through unique genres
for genre in data.tag.unique():
    # Create a subset of the DataFrame for the current genre
    genre_df = data.loc[data['tag'] == genre].drop(columns=['tag'])
    
    # Drop NaN values in 'lyrics_string_fdist'
    genre_df.dropna(subset=['lyrics_string_fdist'], inplace=True)
    
    # Join the 'lyrics_string_fdist' column and tokenize
    genre_lyrics = ' '.join(list(genre_df['lyrics_string_fdist']))
    genre_tokens = word_tokenize(genre_lyrics)
    
    # Calculate frequency distribution
    genre_freq = FreqDist(genre_tokens)
    
    # Store the data in the dictionary
    genre_data[genre] = {'lyrics': genre_lyrics, 'tokens': genre_tokens}
    
    # Update the 'all' category
    genre_data['all']['lyrics'] += genre_lyrics + ' '
    genre_data['all']['tokens'] += genre_tokens

# Calculate frequency distribution for the 'all' category
genre_data['all']['freq'] = FreqDist(genre_data['all']['tokens'])

# Access the frequency distributions for each genre using genre_data
# For example: genre_data['freq_pop'], genre_data['freq_rap'], etc.
# Access the 'all' category: genre_data['all']['freq']
